In [ ]:
import scipp as sc
import plopp as pp
import numpy as np
from matplotlib import pyplot as plt
from math import sqrt
from beamlime.resources import load_icon_img
from beamlime.resources.images.generators import fake_2d_detector_img_generator

%matplotlib widget

In [ ]:
from PIL import Image
from PIL.ImageOps import flip
original_img = Image.fromarray(np.uint8(load_icon_img()))
plt.imshow(Image.fromarray(np.uint8(original_img)))

In [ ]:
# Data Processing Options
detector_size = (64, 64)
x_size = detector_size[0]+1
y_size = detector_size[1]+1

# Fake Data Generating Options
num_frame = 128

In [ ]:
resized_img = original_img.resize(detector_size)
# flip it since image (0, 0) is top-left and histogram plot (0, 0) is top-bottom
seed_img = np.asarray(flip(resized_img), dtype=np.float64)

In [ ]:
def process_img(img, threshold=0.4):
    heatmap = sc.array(dims=['x', 'y'], values=img)
    da = sc.DataArray(data=heatmap, coords={'x': sc.linspace('x', 0, x_size, x_size, unit='mm'),
                                         'y': sc.linspace('y', 0, y_size, y_size, unit='mm')})
    threshold_mask = (da.values > threshold)*np.ones(da.shape)
    da.values *= threshold_mask
    return da

In [ ]:
base = sc.DataArray(data=sc.zeros(sizes={'y': detector_size[1], 'x': detector_size[0]}),
                    coords={'x': sc.linspace('x', 0, x_size, x_size, unit='mm'),
                            'y': sc.linspace('y', 0, y_size, y_size, unit='mm')})

generator = iter(fake_2d_detector_img_generator(seed_img, num_frame=num_frame, threshold=0.5, noise_range=0.55))

for simg in generator:
    sda = process_img(simg, threshold=-float('inf'))
    base.values += sda.values

base.plot(title="Without Filtering")

In [ ]:
a = sda.plot()

In [ ]:
generator = iter(fake_2d_detector_img_generator(seed_img, num_frame=num_frame, threshold=0.5, noise_range=0.55))

for iframe, simg in enumerate(generator):
    sda = process_img(simg, threshold=-float('inf'))
    if iframe < 18:
        ax_x = iframe % 6
        ax_y = iframe // 6
        ax = axs[ax_y][ax_x]
        sda.plot(ax=ax).cax.set(ylabel=None, yticks=[])
        ax.set(xlabel=None, ylabel=None, xticks=[], yticks=[])

In [ ]:
fig, axs = plt.subplots(3, 6, figsize=(12,6))
fig.suptitle('Detector Image Frames To Be Fed - First 18')

In [ ]:
base = sc.DataArray(data=sc.zeros(sizes={'y': detector_size[1], 'x': detector_size[0]}),
                    coords={'x': sc.linspace('x', 0, x_size, x_size, unit='mm'),
                            'y': sc.linspace('y', 0, y_size, y_size, unit='mm')})

generator = iter(fake_2d_detector_img_generator(seed_img, num_frame=num_frame, threshold=0.5, noise_range=0.55))

for simg in generator:
    sda = process_img(simg, threshold=0.6)
    base.values += sda.values

base.plot(title="After Filtering")

In [ ]:
import json
import asyncio
for frame in fake_2d_detector_img_generator(seed_img):
    fake_data = {
        "sample_id": "typical-lime-intaglio-0",
        "detector-data": { {
                "detector-id": "unknown-2d-detector",
                "image": frame.tolist()
            }
        }
    }
    #queue.put(fake_data)